In [1]:
from __future__ import print_function
import matplotlib.pyplot as plt
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
import os
import sys
import zipfile
import urllib.request
import shutil
import numpy as np
import skimage.io as io
import pylab
pylab.rcParams['figure.figsize'] = (10.0, 8.0)

%matplotlib inline


In [2]:
# Record package versions for reproducibility

print("os: %s" % os.name)
print("sys: %s" % sys.version)
print("numpy: %s, %s" % (np.__version__, np.__file__))

notebook_dir = os.path.abspath('')
print("notebook_dir =", notebook_dir)

# project_dir = os.path.abspath(os.path.join(os.path.dirname(notebook_dir), "..", ".."))
project_dir = os.path.abspath("../..")
print(project_dir)
# sys.path.insert(0, project_dir)

os: nt
sys: 3.8.5 (default, Sep  3 2020, 21:29:08) [MSC v.1916 64 bit (AMD64)]
numpy: 1.17.4, D:\Software\anaconda3\envs\rnd\lib\site-packages\numpy\__init__.py
notebook_dir = d:\Workspace\git\RnDProject\cocoapi\PythonAPI\demos
d:\Workspace\git\RnDProject\cocoapi


In [3]:
annType = ['segm','bbox','keypoints']
annType = annType[1]      #specify type here
prefix = 'person_keypoints' if annType=='keypoints' else 'instances'
print('Running demo for *{}* results.'.format(annType))

Running demo for *bbox* results.


In [4]:
# dataDir='../'
# dataType='val2014'
# annFile = '%s/annotations/%s_%s.json'%(dataDir,prefix,dataType)

# Setup data paths
dataDir = os.path.abspath('../../data/coco')
year = "2014"
dataType = 'val{}'.format(year)

annDir = os.path.join(dataDir, 'annotations')
annZipFile = os.path.join(dataDir, 'annotations_trainval{}.zip'.format(year))
annFile = os.path.join(annDir, 'instances_{}.json'.format(dataType))
annURL = 'http://images.cocodataset.org/annotations/annotations_train{}.zip'.format(dataType)
print(annDir)
print(annFile)
print(annZipFile)
print(annURL)


d:\Workspace\git\RnDProject\cocoapi\data\coco\annotations
d:\Workspace\git\RnDProject\cocoapi\data\coco\annotations\instances_val2014.json
d:\Workspace\git\RnDProject\cocoapi\data\coco\annotations_trainval2014.zip
http://images.cocodataset.org/annotations/annotations_trainval2014.zip


In [5]:
# # Download data if not available locally
# if not os.path.exists(annDir):
#     os.makedirs(annDir)
# if not os.path.exists(annFile):
#     if not os.path.exists(annZipFile):
#         print ("Downloading zipped annotations to " + annZipFile + " ...")
#         with urllib.request.urlopen(annURL) as resp, open(annZipFile, 'wb') as out:
#             shutil.copyfileobj(resp, out)
#         print ("... done downloading.")
#     print ("Unzipping " + annZipFile)
#     with zipfile.ZipFile(annZipFile,"r") as zip_ref:
#         zip_ref.extractall(dataDir)
#     print ("... done unzipping")
# print ("Will use annotations in " + annFile)

In [6]:
#initialize COCO ground truth api
cocoGt=COCO(annFile)

loading annotations into memory...
Done (t=5.70s)
creating index...
index created!


In [7]:
#initialize COCO detections api
# resFile='%s/results/%s_%s_fake%s100_results.json'
# resFile = resFile%(dataDir, prefix, dataType, annType)
resFile = os.path.normpath("{}/results/{}_{}_fake{}100_results.json".format(project_dir, prefix, dataType, annType))
print(resFile)
cocoDt=cocoGt.loadRes(resFile)

d:\Workspace\git\RnDProject\cocoapi\results\instances_val2014_fakebbox100_results.json
Loading and preparing results...
DONE (t=0.03s)
creating index...
index created!


In [8]:
imgIds=sorted(cocoGt.getImgIds())
imgIds=imgIds[0:100]
imgId = imgIds[np.random.randint(100)]

In [9]:
# running evaluation
cocoEval = COCOeval(cocoGt,cocoDt,annType)
cocoEval.params.imgIds  = imgIds
cocoEval.evaluate()
cocoEval.accumulate()
cocoEval.summarize()

Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.28s).
Accumulating evaluation results...
DONE (t=0.32s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.505
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.697
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.573
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.586
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.519
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.501
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.387
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.594
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.595
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.640
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100